In [61]:
import numpy as np

# Задание 1

In [62]:
class Neuron:

    def __init__(self, weights_cnt):

        self._weights = np.random.uniform(-1, 1, weights_cnt)
        self._bias = np.random.uniform(-1, 1)
        self._out = 0
        self._sum = 0           


    def __ReLU(self):
        return max(0, self._sum)
    

    def __tanh(self):
        return 2 / (1 + np.exp(-2 * self._sum)) - 1
    

    def __Softmax(self):
        exps = np.exp(self._sum)
        return exps / exps.sum()
    

    def __Sigmoid(self):
        return 1 / (1 + np.exp(-self._sum))
    

    def __deriv_Sigmoid(self, x):
        return x * (1 - x)


    def forward(self, input, fun_activ):

        self._sum = np.dot(input, self._weights) + self._bias

        if fun_activ == 'ReLU':
            self._out = self.__ReLU()
        
        elif fun_activ == 'tanh':
            self._out = self.__tanh()
        
        elif fun_activ == 'Softmax':
            self._out = self.__Softmax()
        
        elif fun_activ == 'Sigmoid':
            self._out = self.__Sigmoid()
        
        return self._out
        
    @property
    def backward(self):
        out = self.__Sigmoid()        
        return self.__deriv_Sigmoid(out)


    def update_weights(self, val):
        self._weights += val


    def update_bias(self, val):
        self._bias += val


    @property
    def get_weights(self):
        return self._weights
    

    @property
    def get_bias(self):
        return self._bias
    
    
    @property
    def get_out(self):
        return self._out

In [63]:
class Model:

    def __init__(self, layers_cnt, cnt_in_layers, funs_activ):
        
        self._layers = [([Neuron(cnt_in_layers[i - 1]) for _ in range(cnt_in_layers[i])], funs_activ[i - 1]) for i in range(1, layers_cnt)]


    def forward(self, input):

        for (layer, fun_activ) in self._layers:
            
            new_input = []

            for neuron in layer:
                new_input.append(neuron.forward(input, fun_activ))

            input = new_input

        return input[0]
    

    def backward(self, input, loss, learn_rate):

        h1 = self._layers[0][0][0]
        h2 = self._layers[0][0][1]
        o1 = self._layers[1][0][0]

        d_w1_h1 = loss * o1.backward * o1.get_weights[0] * h1.backward * input[0]
        d_w2_h1 = loss * o1.backward * o1.get_weights[0] * h1.backward * input[1]
        d_b_h1 = loss * o1.backward * o1.get_weights[0] * h1.backward

        d_w1_h2 = loss * o1.backward * o1.get_weights[1] * h2.backward * input[0]
        d_w2_h2 = loss * o1.backward * o1.get_weights[1] * h2.backward * input[1]
        d_b_h2 = loss * o1.backward * o1.get_weights[1] * h2.backward

        d_w1_o1 = loss * o1.backward * h1.get_out
        d_w2_o1 = loss * o1.backward * h2.get_out
        d_b_o1 = loss * o1.backward

        h1.update_weights([d_w1_h1 * learn_rate, d_w2_h1 * learn_rate])
        h2.update_weights([d_w1_h2 * learn_rate, d_w2_h2 * learn_rate])
        o1.update_weights([d_w1_o1 * learn_rate, d_w2_o1 * learn_rate])

        h1.update_bias(d_b_h1 * learn_rate)
        h2.update_bias(d_b_h2 * learn_rate)
        o1.update_bias(d_b_o1 * learn_rate)
        

    def train(self, train_x, train_y, epochs, learn_rate):

        for i in range(epochs):
  
            id = i % len(train_x)

            y_pred = self.forward(train_x[id])
            self.backward(train_x[id], train_y[id] - y_pred, learn_rate)


    def predict(self, input):

        if self.forward(input) >= 0.5:
            return 1

        return 0

In [64]:
net = Model(5, [256, 64, 16, 4, 1], ['ReLU', 'tanh', 'tanh', 'Softmax'])
print(net.forward(np.random.randint(-100, 100, 256)))

[1.0]


C:\Users\Sanya\AppData\Local\Temp\ipykernel_15084\2881701384.py:16: RuntimeWarning: overflow encountered in exp
  return 2 / (1 + np.exp(-2 * self._sum)) - 1


# Задание 2

In [65]:
class CNN:

    def convolution2D(self, input, kernel):
        
        out_rows_cnt = len(input) - len(kernel) + 1
        out_colms_cnt = len(input[0]) - len(kernel[0]) + 1
        kernel_rows_cnt = len(kernel)
        kernel_colms_cnt = len(kernel[0])

        res = np.zeros((out_rows_cnt, out_colms_cnt))

        for i in range(out_rows_cnt):
            for j in range(out_colms_cnt):

                sum = 0

                for u in range(kernel_rows_cnt):
                    for v in range(kernel_colms_cnt):

                        sum += input[i + u][j + v] * kernel[u][v]

                res[i][j] = sum

        return res


    def convolution_n_channels(self, input, output_size):

        rows_cnt = len(input)
        colms_cnt = len(input[0])
        channels_cnt = len(input[0][0])

        res_channel = np.zeros(output_size)

        for channel in range(channels_cnt):
            
            kernel = np.random.uniform(-1, 1, (rows_cnt - output_size[0] + 1 , colms_cnt - output_size[1] + 1))
            cur_channel = []

            for n in range(rows_cnt):
                for m in range(colms_cnt):
                    cur_channel.append(input[n][m][channel])

            cur_channel = np.array(cur_channel).reshape(rows_cnt, colms_cnt)
            cur_channel = self.convolution2D(cur_channel, kernel)

            res_channel += cur_channel

        return res_channel
    

    def convUp_to_n_channels(self, input, output_size):
        
        conv_cnt = output_size[2]

        res = []

        for i in range(conv_cnt):

            new_channel = self.convolution_n_channels(input, (output_size[0], output_size[1]))
            new_channel = new_channel.ravel()

            for a in range(len(new_channel)):
                res = np.insert(res, i + a * (i + 1), max(new_channel[a], 0)) #Составление результирующей матрицы с использованием ReLU

        return res.reshape(output_size)


    def maxPool(self, input, output_size):
         
        rows_cnt = len(input)
        colms_cnt = len(input[0])
        channels_cnt = len(input[0][0])

        out_w = output_size[0]
        out_h = output_size[1]
        ker_w = input.shape[0] // out_w
        ker_h = input.shape[1] // out_h

        res_mtx = []

        for channel in range(channels_cnt):
         
            cur_channel = []

            for n in range(rows_cnt):
                for m in range(colms_cnt):
                    cur_channel.append(input[n][m][channel])

            cur_channel = np.array(cur_channel).reshape(rows_cnt, colms_cnt)
            cur_channel = cur_channel[:out_w * ker_w, :out_h * ker_h].reshape(out_w, ker_w, out_h, ker_h).max(axis = (1, 3))

            for a in range(len(cur_channel)):
                res_mtx = np.insert(res_mtx, channel + a * (channel + 1), cur_channel[a])

        return res_mtx.reshape(output_size)


    def make_conv_pass(self, img, seq):

        for i in range(0, len(seq) - 1, 2):
            img = self.convUp_to_n_channels(img, seq[i])
            img = self.maxPool(img, seq[i + 1])

        return img

In [66]:
img = np.random.randint(0, 256, (19, 19, 3))
sequence = [(18, 18, 8), (9, 9, 8), (8, 8, 16), (4, 4, 16)]

conv_net = CNN()
img = conv_net.make_conv_pass(img, sequence)

img.shape

(4, 4, 16)

# Задание 3

In [67]:
img = np.random.randint(0, 256, (19, 19, 3))
sequence = [(18, 18, 8), (9, 9, 8), (8, 8, 16), (4, 4, 16), (2, 2, 32), (1, 1, 32)]
net = Model(5, [32, 16, 8, 4, 1], ['ReLU', 'tanh', 'tanh', 'Softmax'])

img = conv_net.make_conv_pass(img, sequence)

print(net.forward(img.ravel()))

[1.0]


C:\Users\Sanya\AppData\Local\Temp\ipykernel_15084\2881701384.py:16: RuntimeWarning: overflow encountered in exp
  return 2 / (1 + np.exp(-2 * self._sum)) - 1


# Лабораторная работа

In [68]:
xor = Model(3, [2, 2, 1], ['Sigmoid', 'Sigmoid'])
xor.train(train_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), train_y = np.array([0, 1, 1, 0]), epochs = 50000, learn_rate = 0.1)

print(xor.predict([0, 0]))
print(xor.predict([0, 1]))
print(xor.predict([1, 0]))
print(xor.predict([1, 1]))

ValueError: non-broadcastable output operand with shape (2,) doesn't match the broadcast shape (2,2)